# ARTIST CENTRALITY _part I_
## Learning from Networks - Project 2022-2023

## Group members : Crivellari Alberto, Khalili Navid, Sartor Nicolò

## Artists closeness: different methods to compute the centrality of nodes in a graph

### 1. Import libraries

In [1]:
import networkx as nx 
import time
import random as rnd
import math
import numpy.random
import pandas
import os

#### 1.1. Create folder of results : *results*

In [2]:
os.mkdir('results')

### 2. Build graph G from nodes in file nodes.csv

In the file *nodes.csv* datas is rappresented in the following format: ___"spotify_id,name,followers,popularity,genres,chart_hits"___.

We are interested in the spotify ID, in the name, in the number of followers and in the popularity index.

### 3. Remove duplicates
In the dataset there are some duplicates so we need to check we are not adding any by mistake and that the one we added are the one with the information we are interested 

(i.e. a duplicate can have the number of followers setted to 0)

N.B.: we need to be carefull splitting the line based on commas since an artist name can have a comma aswell
- without comma: 48WvrUGoijadXXCsGocwM4,Byklubben,1738.0,24,"['nordic house', 'russelater']",['no (3)']

- with comma: 7c1HgFDe8ogy5NOZ1ANCJQ,"Car, the garden",110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"

In [3]:
G = nx.Graph()

#Load dataset with nodes infos
f = open('dataset/nodes.csv', "r", encoding="utf8")

# skip the first line in the input file since it contains dataset description
f.readline()

discarded_counter=0

while True:
    line = f.readline().strip()
    
    #empty line = EOF
    if line == '':
        break
        
    #First we extrapolate the id
    current_id, tmp = line.split(',', 1)
    
    #initialize other variables
    current_artist = current_followers = current_popularity = ''
    
    #Now we divide the 2 cases, artist with comma in their name and artists without,
    #we do that by checking if the first character is equals to '"'
    if tmp[0] != '"':
        #i.e. tmp = Byklubben,1738.0,24,"['nordic house', 'russelater']",['no (3)']
        current_artist, current_followers, current_popularity, tmp = tmp.split(',', 3)
    else:
        #i.e. tmp = "Car, the garden",110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"
        empty, current_artist, tmp = tmp.split('"', 2)
        #i.e. tmp = ,110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"
        empty, current_followers, current_popularity, tmp = tmp.split(',', 3)

    #Now let's try converting followers and popularity index currently string to ints,
    #in case we can't we skip the current node reporting a message
    try:
        current_followers = int(float(current_followers))
        current_popularity = int(current_popularity)                
    except ValueError as ve:
        print('ValueError occured while converting string to int. Node ID:', current_id)
        discarded_counter += 1
        continue
    
    #if the artist is relevant enough then we add him to the graph
    if current_popularity >= 40:
        #if the ID is already a node we check we have the correct informations
        if G.has_node(current_id):
            #if the number of followers of the current line is greater than the numbers of followers already addeed
            #to the graph we just update the corresponding label
            if G.nodes[current_id]['followers'] < current_followers:
                G.nodes[current_id]['followers'] = current_followers
            #same with popularity
            if G.nodes[current_id]['popularity'] < current_popularity:
                G.nodes[current_id]['popularity'] = current_popularity
        
        #else we add it to the graph
        else:
            #add new node with mapped int as key and artist and followers as lables
            G.add_node(current_id, artist=current_artist, followers=current_followers, popularity=current_popularity, path_sum=0)
            
# Close opend file
f.close()
            
#print results
print(discarded_counter,"nodes have been discarded because of bad formatting!")
print(G.number_of_nodes(),"nodes have been added successfully")

ValueError occured while converting string to int. Node ID: 4Jgl9FmNQF6ontIRyY19Ig
ValueError occured while converting string to int. Node ID: 3cCFieWefBXyyDRsjNuArE
ValueError occured while converting string to int. Node ID: 1lLHQcDQFM03FcxZ5mQimA
ValueError occured while converting string to int. Node ID: 7ti7Mdu4BTfKOYWcI1Q6h8
ValueError occured while converting string to int. Node ID: 7estJE1m5cJnQs3Rc4iar0
5 nodes have been discarded because of bad formatting!
28621 nodes have been added successfully


### 4. Add edges to graph G

In the file *edges.csv* datas is rappresented in the following format: ___"id_0,id_1"___.

We need to check the validity of the edge before adding it because some IDs are not on the *nodes.csv* dataset

In [4]:
#Load dataset with edges info
f = open('dataset/edges.csv', "r", encoding="utf8")

# skip the first line in the input file since it contains dataset description
f.readline()

while True:
    line = f.readline().strip()
    
    #empty line = EOF
    if line == '':
        break
        
    #extrapolate id_0 and id_1 from the current line
    id_0, id_1 = line.split(',', 1)
    
    #if the indices are both valid then we add the edge
    if G.has_node(id_0) and G.has_node(id_1):
        G.add_edge(id_0, id_1)
        
# Close opend file
f.close()

print(G.number_of_edges(),"edges have been added successfully")

118141 edges have been added successfully


### 5. Remove nodes not in the main connected component

Since the graph consist in a big main connected component and some really small (in comparisson) connected components/isolated nodes we take in consideration only the main connected component.

To do so we clean up our graph removing those elements disconnected to the main component

In [5]:
print(G.number_of_nodes(),"nodes before clean up")
print(G.number_of_edges(),"edges before clean up")

nodes_main_component = max(nx.connected_components(G), key=len)
G = G.subgraph(nodes_main_component)

print(G.number_of_nodes(),"nodes after clean up")
print(G.number_of_edges(),"edges after clean up")

28621 nodes before clean up
118141 edges before clean up
26389 nodes after clean up
117766 edges after clean up


### 6. Exact Closeness Centrality for whole graph <b>[3hours and half]</b>

We keep track of the computation time 

In [6]:
start_time = time.time()
exact_closeness_centrality = nx.closeness_centrality(G)
end_time = time.time()

#let's sort the results based on the valu
exact_closeness_centrality = {k: v for k, v in sorted(exact_closeness_centrality.items(),reverse=True , key=lambda item: item[1])}

#let's print the results
#print(exact_closeness_centrality)
print("the exact closeness centralities have been computed in %s seconds" %(end_time - start_time))

#Now let's store the results
f = open('results/results_exact_closeness_centrality.txt', "w", encoding="utf8")
for key, value in exact_closeness_centrality.items():
    f.write('%s:%s\n' %(key, value))

# Close opend file
f.close()

the exact closeness centralities have been computed in 5606.1627860069275 seconds


### 7. Functions for Approximated Closeness Centrality

#### 7.1. Eppstein-Wang Algorithm

Approximated closeness centrality with uniform sampling

In [7]:
def ApproximateClosenessCentrality_EW(G, k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0
        
    for i in range(k):
        #pick one node uniformally at random
        random_node = rnd.choice(list(G.nodes()))
        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities      

#### 7.2. Chechik-Cohen-Kaplan Algorithm

Approximated closeness centrality with Poisson sampling

In [8]:
def ApproximateClosenessCentrality_CCK(G,k):
    for n in G:
        G.nodes[n]['path_sum'] = 0
        G.nodes[n]['pv']=1/G.number_of_nodes()

    sample_S0 = []
    size_S0 = int(k/100)+1

    for i in range(size_S0):
        sample_S0.append(rnd.choice(list(G.nodes())))
    for s in sample_S0 :
        W = 0
        for n in G:
            G.nodes[n]['path_sum'] = 0
        sssp = nx.shortest_path_length(G, source=s)
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
        for n in G :
            W = W + G.nodes[n]['path_sum']
        for n in G :
            G.nodes[n]['pv'] = max(G.nodes[n]['pv'], G.nodes[n]['path_sum']/W)
    for n in G:
        G.nodes[n]['pv'] = min(1, k*G.nodes[n]['pv'])
    
    final_sample_S = []
    centralities = {}
    
    for n in G :
        centralities[n] = 0
        if numpy.random.uniform(0,1) < G.nodes[n]['pv'] :
            final_sample_S.append(n)
    
    for u in final_sample_S :
        for n in G:
            G.nodes[n]['path_sum'] = 0
        sssp = nx.shortest_path_length(G, source=s)
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
        for n in G :
            centralities[n] = centralities[n] + (G.nodes[n]['path_sum']/G.nodes[n]['pv'])
    
    for n in G :
        if centralities[n] != 0 :
            centralities[n] = 1/((G.number_of_nodes()*centralities[n])/(len(final_sample_S)*(G.number_of_nodes()-1)))
    
    #return dicionary containg pairs (node, approximated_centrality)
    return centralities    

#### 7.3. Approximated Closeness Centrality based on Node Degree

Approximation with a weighted probability(instead of uniform probability) based on the degree of each node

In [9]:
def ApproximateClosenessCentrality_NodeDegree(G, k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0
        
    #make a list of degree of nodes. The second elements of the list "G.degree" shows the degree of each node
    nodes_degree = [x[1] for x in G.degree]
    
    for i in range(k):
        #pick one node at random with weights based on the degree of the nodes 
        random_node = rnd.choices(list(G.nodes()), weights=list(nodes_degree), k=1)[0]

        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities 

### 8. Computation of Approximated Closeness Centrality
We will compute approximated closeness centralities with 2 different k, one with epsilon 0.1 and another k with epsilon 0.05

For each k we will compute approximate closeness centralities with our 3 algorithms: Eppstein-Wang, Chechik-Cohen-Kaplan and the approximation algorithm based on Node Degree.
#### 8.1. $\varepsilon = 0.1$
We use the formula:  $k = \frac{log(n)}{\varepsilon ^2}$

In [10]:
epsilon = 0.1
k = int(math.log(G.number_of_nodes(),2)/(epsilon**2))

##### 8.1.1. Computation using Eppstein-Wang Algorithm 

In [11]:
start_time = time.time()
approximated_closeness_centrality_EW_1 = ApproximateClosenessCentrality_EW(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_EW_1 = {k: v for k, v in sorted(approximated_closeness_centrality_EW_1.items(), reverse=True, key=lambda item: item[1])}

#let's print the results
#print(approximated_closeness_centrality_EW_1)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_EW_epsilon_0_1.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_EW_1.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opened file
f.close()

the approximated closeness centralities with 1468 iterations have been computed in 754.5324695110321 seconds


##### 8.1.2. Computation using Chechik-Cohen-Kaplan Algorithm

In [12]:
start_time = time.time()
approximated_closeness_centrality_CCK_1 = ApproximateClosenessCentrality_CCK(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_CCK_1 = {k: v for k, v in sorted(approximated_closeness_centrality_CCK_1.items(), reverse=True, key=lambda item: item[1])}

#let's print the results
#print(approximated_closeness_centrality_CCK_1)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_CCK_epsilon_0_1.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_CCK_1.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opened file
f.close()

the approximated closeness centralities with 1468 iterations have been computed in 1072.6708309650421 seconds


##### 8.1.3. Computation using Approximation Algorithm based on Node Degree

In [13]:
start_time = time.time()
approximated_closeness_centrality_NodeDegree_1 = ApproximateClosenessCentrality_NodeDegree(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_NodeDegree_1 = {k: v for k, v in sorted(approximated_closeness_centrality_NodeDegree_1.items(), reverse=True, key=lambda item: item[1])}

#let's print the results
#print(approximated_closeness_centrality_NodeDegree_1)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_NodeDegree_epsilon_0_1.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_NodeDegree_1.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opened file
f.close()

the approximated closeness centralities with 1468 iterations have been computed in 758.7651515007019 seconds


#### 8.2.  $\varepsilon = 0.05$

In [14]:
epsilon = 0.05
k = int(math.log(G.number_of_nodes(),2)/(epsilon**2))

##### 8.2.1. Computation using Eppstein-Wang Algorithm 

In [15]:
start_time = time.time()
approximated_closeness_centrality_EW_2 = ApproximateClosenessCentrality_EW(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_EW_2 = {k: v for k, v in sorted(approximated_closeness_centrality_EW_2.items(), reverse=True, key=lambda item: item[1])}

#let's print the results
#print(approximated_closeness_centrality_EW_2)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_EW_epsilon_0_05.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_EW_2.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opened file
f.close()

the approximated closeness centralities with 5875 iterations have been computed in 1867.2176263332367 seconds


##### 8.2.2. Computation using Chechik-Cohen-Kaplan Algorithm 

In [16]:
start_time = time.time()
approximated_closeness_centrality_CCK_2 = ApproximateClosenessCentrality_CCK(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_CCK_2 = {k: v for k, v in sorted(approximated_closeness_centrality_CCK_2.items(), reverse=True, key=lambda item: item[1])}

#let's print the results
#print(approximated_closeness_centrality_CCK_2)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_CCK_epsilon_0_05.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_CCK_2.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opened file
f.close()

the approximated closeness centralities with 5875 iterations have been computed in 2762.262521982193 seconds


##### 8.2.3. Computation using Approximation Algorithm based on Node Degree

In [17]:
start_time = time.time()
approximated_closeness_centrality_NodeDegree_2 = ApproximateClosenessCentrality_NodeDegree(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_NodeDegree_2 = {k: v for k, v in sorted(approximated_closeness_centrality_NodeDegree_2.items(), reverse=True, key=lambda item: item[1])}

#let's print the results
#print(approximated_closeness_centrality_NodeDegree_2)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_NodeDegree_epsilon_0_05.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_NodeDegree_2.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opened file
f.close()

the approximated closeness centralities with 5875 iterations have been computed in 1882.1182265281677 seconds


#### 9. Compute statistics from our closeness centralities

So we computed the closeness centralities of exact algorithm and from approximated algorithms, the 3 algorithms with epsilon 0.1 and 3 with epsilon 0.05

So we have 7 arrays of closeness centralities results and now we try to compute some statistics out of them:
- mean exact closeness centrality
- mean approximated closeness centrality
- mean error
- standard deviations
- top 10 artist using exact closeness centrality
- top 10 artist using approximated closeness centrality

In [18]:
start_time = time.time()

closeness_centralities = []
closeness_centralities.append(exact_closeness_centrality)
closeness_centralities.append(approximated_closeness_centrality_EW_1)
closeness_centralities.append(approximated_closeness_centrality_CCK_1)
closeness_centralities.append(approximated_closeness_centrality_NodeDegree_1)
closeness_centralities.append(approximated_closeness_centrality_EW_2)
closeness_centralities.append(approximated_closeness_centrality_CCK_2)
closeness_centralities.append(approximated_closeness_centrality_NodeDegree_2)

mean = []
mean_errors = []
std_dev = []
std_dev_errors = []
max_absolute_errors = []

for i in range(6) :
    mean_errors.append(0)
    std_dev_errors.append(0)
    max_absolute_errors.append(0)
for i in range(7) :
    mean.append(0)
    std_dev.append(0)

# Let's compute means and mean errors, and max absolute errors
for n in G :
    for i in range(7):
        mean[i] += closeness_centralities[i][n]
        if i > 0 :
            actual_error = closeness_centralities[0][n] - closeness_centralities[i][n]
            if max_absolute_errors[i-1] < abs(actual_error) :
                max_absolute_errors[i-1] = abs(actual_error)
for i in range(7) :
    mean[i] = mean[i]/G.number_of_nodes()
    if i > 0 :
        mean_errors[i-1] = mean[0] - mean[i]

# Let's compute std devs and std dev errors
for n in G :
    for i in range(7):
        std_dev[i] = (closeness_centralities[i][n] - mean[i])**2
        if i > 0 :
            std_dev_errors[i-1] = (closeness_centralities[0][n] - closeness_centralities[i][n] - mean_errors[i-1])**2
for i in range(7) :
    std_dev[i] = (std_dev[i]/G.number_of_nodes())**0.5
    if i > 0 :
        std_dev_errors[i-1] = (std_dev_errors[i-1]/G.number_of_nodes())**0.5

        
end_time = time.time()
print("The statistics of the closeness centralities have been computed in %s seconds" %(end_time - start_time))


The statistics of the closeness centralities have been computed in 0.4238560199737549 seconds


##### 9.1. Store statistics in a file
Let's now store the results in a file called *statistics.txt* inside the folder *results*

In [19]:
#Let's store the results

stats_file = open('results/statistics.txt', "w", encoding="utf8")
stats_file.write(f"Here we have the statistics of these 7 algorithms for computing closeness centralities on the whole graph:\n1: Exact Algorithm\n2: Eppstein-Wang Algorithm with epsilon = 0.1\n3: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.1\n4: Node Degree Algorithm with epsilon = 0.1\n5: Eppstein-Wang Algorithm with epsilon = 0.05\n6: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.05\n7: Node Degree Algorithm with epsilon = 0.05\n\n")

stats_file.write(f"Means:\n")
stats_file.write(f"1: {mean[0]}\n")
stats_file.write(f"2: {mean[1]}\n")
stats_file.write(f"3: {mean[2]}\n")
stats_file.write(f"4: {mean[3]}\n")
stats_file.write(f"5: {mean[4]}\n")
stats_file.write(f"6: {mean[5]}\n")
stats_file.write(f"7: {mean[6]}\n\n")

stats_file.write(f"Standard Deviations:\n")
stats_file.write(f"1: {std_dev[0]}\n")
stats_file.write(f"2: {std_dev[1]}\n")
stats_file.write(f"3: {std_dev[2]}\n")
stats_file.write(f"4: {std_dev[3]}\n")
stats_file.write(f"5: {std_dev[4]}\n")
stats_file.write(f"6: {std_dev[5]}\n")
stats_file.write(f"7: {std_dev[6]}\n\n")

stats_file.write(f"Maximum Absolute Errors between Exact Algorithm and Approximated:\n")
stats_file.write(f"1-2: {max_absolute_errors[0]}\n")
stats_file.write(f"1-3: {max_absolute_errors[1]}\n")
stats_file.write(f"1-4: {max_absolute_errors[2]}\n")
stats_file.write(f"1-5: {max_absolute_errors[3]}\n")
stats_file.write(f"1-6: {max_absolute_errors[4]}\n")
stats_file.write(f"1-7: {max_absolute_errors[5]}\n\n")

stats_file.write(f"Mean Errors between Exact Algorithm and Approximated:\n")
stats_file.write(f"1-2: {mean_errors[0]}\n")
stats_file.write(f"1-3: {mean_errors[1]}\n")
stats_file.write(f"1-4: {mean_errors[2]}\n")
stats_file.write(f"1-5: {mean_errors[3]}\n")
stats_file.write(f"1-6: {mean_errors[4]}\n")
stats_file.write(f"1-7: {mean_errors[5]}\n\n")

stats_file.write(f"Std Dev Errors between Exact Algorithm and Approximated:\n")
stats_file.write(f"1-2: {std_dev_errors[0]}\n")
stats_file.write(f"1-3: {std_dev_errors[1]}\n")
stats_file.write(f"1-4: {std_dev_errors[2]}\n")
stats_file.write(f"1-5: {std_dev_errors[3]}\n")
stats_file.write(f"1-6: {std_dev_errors[4]}\n")
stats_file.write(f"1-7: {std_dev_errors[5]}\n\n")

# Close opend file
stats_file.close()
    

#### 9.2. Print statistics as a table

Insert in errors lists as the first element the character '\\': since we compare the errors between algorithm 0 (the exact algorithm) and the other algorithms, we don't have an error between 0 algorithm and 0 algorithm (it would be 0)

In [20]:
mean_errors.insert(0,'\\')
max_absolute_errors.insert(0,'\\')
std_dev_errors.insert(0,'\\')

Now we create the table and print it

In [21]:
table = {'Means': mean, 'Standard Deviations' : std_dev, 'Mean Errors (0vsX)' : mean_errors, 'Maximum Absolute Errors (1vsX)' : max_absolute_errors, 'Standard Deviation Errors (1vsX)' : std_dev_errors}
algorithms_names = ['1: Exact Algorithm', '2: Eppstein-Wang Algorithm with epsilon = 0.1', '3: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.1', '4: Node Degree Algorithm with epsilon = 0.1', '5: Eppstein-Wang Algorithm with epsilon = 0.05', '6: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.05', '7: Node Degree Algorithm with epsilon = 0.05']
pandas.DataFrame(data = table, index = algorithms_names)


,Means,Standard Deviations,Mean Errors (0vsX),Maximum Absolute Errors (1vsX),Standard Deviation Errors (1vsX)
1: Exact Algorithm,0.197039,0.000047,\,\,\
2: Eppstein-Wang Algorithm with epsilon = 0.1,0.197980,0.000046,-0.000941,0.004072,0.000001
3: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.1,0.011582,0.000005,0.185457,0.29001,0.000052
4: Node Degree Algorithm with epsilon = 0.1,0.229627,0.000062,-0.032589,0.087366,0.000015
5: Eppstein-Wang Algorithm with epsilon = 0.05,0.197557,0.000046,-0.000518,0.00229,0.000001
6: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.05,0.054549,0.000024,0.14249,0.245553,0.000071
7: Node Degree Algorithm with epsilon = 0.05,0.229354,0.000064,-0.032315,0.086485,0.000016


In [22]:
table = {'Means': mean, 'Standard Deviations' : std_dev, 'Mean Errors (0vsX)' : mean_errors, 'Maximum Absolute Errors (1vsX)' : max_absolute_errors, 'Standard Deviation Errors (1vsX)' : std_dev_errors}
algorithms_names = ['1: Exact Algorithm', '2: Eppstein-Wang Algorithm with epsilon = 0.1', '3: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.1', '4: Node Degree Algorithm with epsilon = 0.1', '5: Eppstein-Wang Algorithm with epsilon = 0.05', '6: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.05', '7: Node Degree Algorithm with epsilon = 0.05']

print('1: Exact Algorithm')
print('2: Eppstein-Wang Algorithm with epsilon = 0.1')
print('3: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.1')
print('4: Node Degree Algorithm with epsilon = 0.1')
print('5: Eppstein-Wang Algorithm with epsilon = 0.05')
print('6: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.05')
print('7: Node Degree Algorithm with epsilon = 0.05\n\n')
pandas.DataFrame(data = table, index = [1, 2, 3, 4, 5, 6, 7])

1: Exact Algorithm
2: Eppstein-Wang Algorithm with epsilon = 0.1
3: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.1
4: Node Degree Algorithm with epsilon = 0.1
5: Eppstein-Wang Algorithm with epsilon = 0.05
6: Chechik-Cohen-Kaplan Algorithm with epsilon = 0.05
7: Node Degree Algorithm with epsilon = 0.05




,Means,Standard Deviations,Mean Errors (0vsX),Maximum Absolute Errors (1vsX),Standard Deviation Errors (1vsX)
1,0.197039,0.000047,\,\,\
2,0.197980,0.000046,-0.000941,0.004072,0.000001
3,0.011582,0.000005,0.185457,0.29001,0.000052
4,0.229627,0.000062,-0.032589,0.087366,0.000015
5,0.197557,0.000046,-0.000518,0.00229,0.000001
6,0.054549,0.000024,0.14249,0.245553,0.000071
7,0.229354,0.000064,-0.032315,0.086485,0.000016
